In [12]:
import socket
import json
import threading
import time


# Servidor

def start_server():
    host = "127.0.0.1"
    port = 5050
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind((host, port))
    server_socket.listen(1)

    print(f"Servidor aguardando conexões em {host}:{port}...")
    conn, addr = server_socket.accept()
    print(f"Cliente conectado: {addr}")

    questions = [
        {
            "question": "Qual é a capital da Itália?",
            "options": ["Roma", "Paris", "Lisboa", "Londres"],
            "answer": "Roma"
        },
        {
            "question": "Qual é o maior planeta do Sistema Solar?",
            "options": ["Terra", "Júpiter", "Saturno", "Marte"],
            "answer": "Júpiter"
        }
    ]

    conn.send(json.dumps(questions).encode())
    data = conn.recv(1024).decode()
    answers = json.loads(data)

    results, score = [], 0
    for q, resp in zip(questions, answers):
        if resp == q["answer"]:
            results.append({"question": q["question"], "resultado": "Acertou"})
            score += 1
        else:
            results.append({"question": q["question"], "resultado": "Errou"})

    feedback = {"score": score, "results": results}
    conn.send(json.dumps(feedback).encode())
    conn.close()


# Cliente

def start_client():
    time.sleep(1)  # espera servidor iniciar
    host = "127.0.0.1"
    port = 5050

    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))

    data = client_socket.recv(1024).decode()
    questions = json.loads(data)

    answers = []
    for q in questions:
        print("\n" + q["question"])
        for i, opt in enumerate(q["options"]):
            print(f"{i+1}) {opt}")
        # Como não dá para digitar interativamente no Colab enquanto os dois rodam,
        # aqui estou simulando: sempre escolhe a primeira opção
        choice = 1
        answers.append(q["options"][choice-1])

    client_socket.send(json.dumps(answers).encode())

    feedback = client_socket.recv(1024).decode()
    feedback = json.loads(feedback)

    print("\n--- RESULTADO FINAL ---")
    print(f"Acertos: {feedback['score']}/{len(questions)}")
    for r in feedback["results"]:
        print(f"{r['question']} -> {r['resultado']}")

    client_socket.close()

# Executar servidor e cliente

t1 = threading.Thread(target=start_server)
t2 = threading.Thread(target=start_client)

t1.start()
t2.start()

t1.join()
t2.join()


Exception in thread Thread-4 (start_client):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-329993434.py", line 65, in start_client
ConnectionResetError: [Errno 104] Connection reset by peer
Exception in thread Thread-6 (start_client):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-2884738516.py", line 12, in start_client
ConnectionResetError: [Errno 104] Connection reset by peer


Servidor aguardando conexões em 127.0.0.1:5050...
Cliente conectado: ('127.0.0.1', 41856)

Qual é a capital da Itália?
1) Roma
2) Paris
3) Lisboa
4) Londres

Qual é o maior planeta do Sistema Solar?
1) Terra
2) Júpiter
3) Saturno
4) Marte

--- RESULTADO FINAL ---
Acertos: 1/2
Qual é a capital da Itália? -> Acertou
Qual é o maior planeta do Sistema Solar? -> Errou
